### **GPU 설정**

In [100]:
import tensorflow as tf

# GPU 디바이스 확인
tf.test.gpu_device_name()

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# TensorFlow가 GPU를 사용할 수 있도록 설정합니다.
if len(tf.config.list_physical_devices('GPU')) > 0:
    print('GPU is enabled')
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

Num GPUs Available:  1
GPU is enabled


### **구글 드라이브 마운트**

In [101]:
from google.colab import drive

# 드라이브 마운트
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **학습 데이터 불러오기**

In [102]:
import numpy as np
import os

# 저장된 경로
load_path = '/content/drive/MyDrive/ColabNotebooks/프로젝트/WordDifficulty/data'

# 넘파이 배열 불러오기
X_train = np.load(os.path.join(load_path, 'X_train.npy'), allow_pickle=True)
y_train = np.load(os.path.join(load_path, 'y_train.npy'), allow_pickle=True)
X_val = np.load(os.path.join(load_path, 'X_val.npy'), allow_pickle=True)
y_val = np.load(os.path.join(load_path, 'y_val.npy'), allow_pickle=True)
X_test = np.load(os.path.join(load_path, 'X_test.npy'), allow_pickle=True)
y_test = np.load(os.path.join(load_path, 'y_test.npy'), allow_pickle=True)

import re
# 정규식을 사용하여 데이터 정리
X_train = np.array([re.sub(r"[^a-zA-Z]", " ", str(x)) for x in X_train])
X_val = np.array([re.sub(r"[^a-zA-Z]", " ", str(x)) for x in X_val])
X_test = np.array([re.sub(r"[^a-zA-Z]", " ", str(x)) for x in X_test])


# 결과 확인
print("Loaded X_train shape:", X_train.shape)
print("Loaded y_train shape:", y_train.shape)
print("Loaded X_val shape:", X_val.shape)
print("Loaded y_val shape:", y_val.shape)
print("Loaded X_test shape:", X_test.shape)
print("Loaded y_test shape:", y_test.shape)


Loaded X_train shape: (28336,)
Loaded y_train shape: (28336,)
Loaded X_val shape: (6072,)
Loaded y_val shape: (6072,)
Loaded X_test shape: (6073,)
Loaded y_test shape: (6073,)


In [103]:
print(X_train)

['Alice' 'finality' 'showery' ... 'unmagnified' 'Alabama' 'groat']


In [104]:
print(y_train)

[-0.46 -0.18 -0.01 ...  0.93 -0.4  -0.3 ]


In [105]:
from gensim.models import KeyedVectors

# Word2Vec 모델 로드
model_path = '/content/drive/MyDrive/ColabNotebooks/프로젝트/WordDifficulty/GoogleNews-vectors-negative300.bin'  # 모델 파일 경로
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)


In [99]:
# 예시: 단어 벡터 확인
word_vector = word2vec_model['house']
print(word_vector)

[-1.0331274  -1.1010782   0.7369398  -0.01692823  0.6463364  -2.3608403
  0.3288975  -0.04326618  0.5647297   0.06302029 -1.8109397  -0.18449123
 -0.03135364 -0.15083504  0.23495762 -0.216176    0.29746395 -0.33901358
 -0.11315782 -0.4308508   0.44912028 -0.158004   -1.6367283  -0.07075656
  1.7458962  -0.3922746  -0.03588879  1.2564448   1.5550973  -0.7053086
 -2.3644443   1.6595973  -1.5981705  -1.3482987  -1.1281877   0.5220224
  1.9134964  -0.73305506 -1.2080895  -0.52565813  1.2057009  -0.36146933
 -0.40767944 -0.3714348   1.5670446  -0.75321794 -0.55204153  0.1601412
 -0.1419861   0.5581101  -1.2331018  -1.7637047  -0.5457022  -0.48225817
 -0.47613773 -0.3606465  -0.40019637 -0.16795826 -0.8913736  -0.28229392
 -0.4967095  -0.7084877   1.2193099   1.1518924   0.1828406  -0.08150962
  0.90322983  0.03852229  1.666108    0.5499143  -0.3505481   1.1958036
  1.3369023   0.65769494 -0.38484883 -0.20659938  0.84080577 -1.1813729
 -0.6393107   1.2325064  -0.22848767 -1.2068859   0.00384

In [106]:
def remove_invalid_vectors(data, word2vec_model):
    valid_indices = []
    for i, word in enumerate(data):
        if word not in word2vec_model.key_to_index:
            print("유효하지 않은 벡터 존재:", word)
        else:
            valid_indices.append(i)
    return data[valid_indices], valid_indices

def convert_words_to_vectors(data, word2vec_model):
    vectors = []
    for word in data:
        if word in word2vec_model.key_to_index:
            vector = word2vec_model[word]
            # 벡터의 모든 원소가 0부터 1 사이의 값을 가지도록 정규화
            normalized_vector = (vector - np.min(vector)) / (np.max(vector) - np.min(vector))
            vectors.append(normalized_vector)
        else:
            print("유효하지 않은 벡터 존재:", word)
    return np.array(vectors)


# 유효하지 않은 벡터 제거
X_train, train_valid_indices = remove_invalid_vectors(X_train, word2vec_model)
y_train = y_train[train_valid_indices]

X_val, val_valid_indices = remove_invalid_vectors(X_val, word2vec_model)
y_val = y_val[val_valid_indices]

X_test, test_valid_indices = remove_invalid_vectors(X_test, word2vec_model)
y_test = y_test[test_valid_indices]

# 단어를 벡터로 변환
X_train_vectors = convert_words_to_vectors(X_train, word2vec_model)
X_val_vectors = convert_words_to_vectors(X_val, word2vec_model)
X_test_vectors = convert_words_to_vectors(X_test, word2vec_model)


유효하지 않은 벡터 존재: queensberry
유효하지 않은 벡터 존재: Brandon s
유효하지 않은 벡터 존재: keelhaul
유효하지 않은 벡터 존재: marine s
유효하지 않은 벡터 존재: printer s
유효하지 않은 벡터 존재: reveller
유효하지 않은 벡터 존재: pleasance
유효하지 않은 벡터 존재: mineralogies
유효하지 않은 벡터 존재: Lindsey s
유효하지 않은 벡터 존재: season s
유효하지 않은 벡터 존재: dactyls
유효하지 않은 벡터 존재: when ll
유효하지 않은 벡터 존재: paper s
유효하지 않은 벡터 존재: fourpence
유효하지 않은 벡터 존재: Arianists
유효하지 않은 벡터 존재: Washington s
유효하지 않은 벡터 존재: pastness
유효하지 않은 벡터 존재: fullbacking
유효하지 않은 벡터 존재: client s
유효하지 않은 벡터 존재: teacart
유효하지 않은 벡터 존재: parent s
유효하지 않은 벡터 존재: dreamt
유효하지 않은 벡터 존재: preordainment
유효하지 않은 벡터 존재: Michelangelo s
유효하지 않은 벡터 존재: governor s
유효하지 않은 벡터 존재: economist s
유효하지 않은 벡터 존재: dissolubility
유효하지 않은 벡터 존재: Colorado s
유효하지 않은 벡터 존재: halfpence
유효하지 않은 벡터 존재: Shelley s
유효하지 않은 벡터 존재: table s
유효하지 않은 벡터 존재: infelicity
유효하지 않은 벡터 존재: footlight
유효하지 않은 벡터 존재: hayfork
유효하지 않은 벡터 존재: hurrays
유효하지 않은 벡터 존재: millwheel
유효하지 않은 벡터 존재: what re
유효하지 않은 벡터 존재: expurgation
유효하지 않은 벡터 존재: Ann s
유효하지 않은 벡터 존재: clothesbas

In [107]:
# 결과 확인
print("Processed X_train shape:", X_train.shape)
print("Processed y_train shape:", y_train.shape)
print("Processed X_val shape:", X_val.shape)
print("Processed y_val shape:", y_val.shape)
print("Processed X_test shape:", X_test.shape)
print("Processed y_test shape:", y_test.shape)

Processed X_train shape: (26929,)
Processed y_train shape: (26929,)
Processed X_val shape: (5743,)
Processed y_val shape: (5743,)
Processed X_test shape: (5750,)
Processed y_test shape: (5750,)


In [108]:
import numpy as np

# NaN 값 확인 함수
def check_nan_values(array):
    return np.isnan(array).any()

# NaN 값이 있는 인덱스 찾기
nan_train_indices = np.isnan(y_train)
nan_test_indices = np.isnan(y_test)
nan_val_indices = np.isnan(y_val)

# NaN 값이 없는 인덱스 찾기
valid_train_indices = np.logical_not(nan_train_indices)
valid_test_indices = np.logical_not(nan_test_indices)
valid_val_indices = np.logical_not(nan_val_indices)

# NaN 값을 제거한 데이터 구성
X_train_valid = X_train_vectors[valid_train_indices]
y_train_valid = y_train[valid_train_indices]

X_test_valid = X_test_vectors[valid_test_indices]
y_test_valid = y_test[valid_test_indices]

X_val_valid = X_val_vectors[valid_val_indices]
y_val_valid = y_val[valid_val_indices]

# NaN 값을 삭제한 X_train_vectors, X_test_vectors, X_val_vectors 구성
X_train_vectors_valid = X_train_vectors[valid_train_indices]
X_test_vectors_valid = X_test_vectors[valid_test_indices]
X_val_vectors_valid = X_val_vectors[valid_val_indices]

# 결과 출력
print("NaN values in X_train_vectors:", check_nan_values(X_train_vectors_valid))
print("NaN values in y_train:", check_nan_values(y_train_valid))
print("NaN values in X_test_vectors:", check_nan_values(X_test_vectors_valid))
print("NaN values in y_test:", check_nan_values(y_test_valid))
print("NaN values in X_val_vectors:", check_nan_values(X_val_vectors_valid))
print("NaN values in y_val:", check_nan_values(y_val_valid))

# 수정된 데이터의 형태 출력
print("Modified X_train_vectors shape:", X_train_valid.shape)
print("Modified y_train shape:", y_train_valid.shape)
print("Modified X_test_vectors shape:", X_test_valid.shape)
print("Modified y_test shape:", y_test_valid.shape)
print("Modified X_val_vectors shape:", X_val_valid.shape)
print("Modified y_val shape:", y_val_valid.shape)


NaN values in X_train_vectors: False
NaN values in y_train: False
NaN values in X_test_vectors: False
NaN values in y_test: False
NaN values in X_val_vectors: False
NaN values in y_val: False
Modified X_train_vectors shape: (26923, 300)
Modified y_train shape: (26923,)
Modified X_test_vectors shape: (5749, 300)
Modified y_test shape: (5749,)
Modified X_val_vectors shape: (5742, 300)
Modified y_val shape: (5742,)


In [109]:
import numpy as np
from tensorflow import keras

#모델 아키텍처 정의
model = keras.models.Sequential()
model.add(keras.layers.Dense(units=512, activation='relu', input_shape=(300,))) # 첫 번째 은닉층
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=32, activation='relu'))
model.add(keras.layers.Dense(units=1)) # 출력 레이어 (회귀 모델이므로 활성화 함수 없음)

#모델 컴파일
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#모델 훈련
history = model.fit(X_train_vectors_valid, y_train_valid, validation_data=(X_val_vectors_valid, y_val_valid), epochs=20, batch_size=32)

Epoch 1/20
842/842 [==============================] - 6s 5ms/step - loss: 0.1629 - mean_squared_error: 0.1629 - val_loss: 0.1348 - val_mean_squared_error: 0.1348
Epoch 2/20
842/842 [==============================] - 5s 6ms/step - loss: 0.1319 - mean_squared_error: 0.1319 - val_loss: 0.1308 - val_mean_squared_error: 0.1308
Epoch 3/20
842/842 [==============================] - 4s 5ms/step - loss: 0.1301 - mean_squared_error: 0.1301 - val_loss: 0.1269 - val_mean_squared_error: 0.1269
Epoch 4/20
842/842 [==============================] - 4s 5ms/step - loss: 0.1267 - mean_squared_error: 0.1267 - val_loss: 0.1231 - val_mean_squared_error: 0.1231
Epoch 5/20
842/842 [==============================] - 4s 5ms/step - loss: 0.1253 - mean_squared_error: 0.1253 - val_loss: 0.1222 - val_mean_squared_error: 0.1222
Epoch 6/20
842/842 [==============================] - 5s 6ms/step - loss: 0.1261 - mean_squared_error: 0.1261 - val_loss: 0.1185 - val_mean_squared_error: 0.1185
Epoch 7/20
842/842 [========

In [110]:
# 예측 수행
y_pred = model.predict(X_test_vectors)

# MSE 계산
mse = np.mean(np.square(y_test - y_pred))

# 결과 출력
print('Mean Squared Error:', mse)


180/180 [==============================] - 0s 2ms/step
Mean Squared Error: nan


In [125]:
import random


# Word2Vec 모델에서 학습된 모든 단어
all_words = word2vec_model.key_to_index.keys()

# 임의의 단어 선택
random_word = random.choice(list(all_words))

# 선택한 단어의 벡터 출력
if random_word in word2vec_model:
    input_vector = word2vec_model[random_word]
    input_vector = np.reshape(input_vector, (1, -1))
    output_vector = model.predict(input_vector)
    print('Input word:', random_word)
    print('Output vector:', output_vector)
else:
    print('Invalid word:', random_word)

1/1 [==============================] - 0s 18ms/step
Input word: Vahlkamp
Output vector: [[0.3146061]]


In [127]:
# 벡터 입력
input_vector = word2vec_model['house']

# 입력 데이터를 2차원으로 reshape
input_vector = np.reshape(input_vector, (1, 300))

print(input_vector)

# 모델 예측
output_vector = model.predict(input_vector)

# 결과 출력
print('Output vector:', output_vector)

[[ 1.57226562e-01 -7.08007812e-02  5.39550781e-02 -1.89208984e-02
   9.17968750e-02  2.55126953e-02  7.37304688e-02 -5.68847656e-02
   1.79687500e-01  9.27734375e-02  9.03320312e-02 -4.12109375e-01
  -8.30078125e-02 -1.45507812e-01 -2.37304688e-01 -3.68652344e-02
   8.74023438e-02 -2.77099609e-02  1.13677979e-03  8.30078125e-02
   3.57421875e-01 -2.61718750e-01  7.47070312e-02 -8.10546875e-02
  -2.35595703e-02 -1.61132812e-01 -4.78515625e-02  1.85546875e-01
  -3.97949219e-02 -1.58203125e-01 -4.37011719e-02 -1.11328125e-01
  -1.05957031e-01  9.86328125e-02 -8.34960938e-02 -1.27929688e-01
  -1.39648438e-01 -1.86523438e-01 -5.71289062e-02 -1.17675781e-01
  -1.32812500e-01  1.55639648e-02  1.34765625e-01  8.39843750e-02
  -9.03320312e-02 -4.12597656e-02 -2.51953125e-01 -2.27539062e-01
  -6.64062500e-02 -7.66601562e-02  5.15136719e-02  5.90820312e-02
   3.49609375e-01 -1.13769531e-01 -2.57568359e-02 -1.98242188e-01
   4.44335938e-02  1.09863281e-01  1.04003906e-01 -1.75781250e-01
   1.22558

In [128]:
model.save("model.h5")
